#### Author: Nagibe Maroun González

# Jupyter notebook for downloading Surface Albedo data

Surface albedo is critical for understanding the Earth’s climate, its variability, and trends, as it quantifies the fraction of solar irradiance reflected by the Earth’s surface. This variable plays a vital role in the global radiation budget, influencing surface temperatures, water balance, and climate feedback mechanisms, such as the ice-albedo feedback in polar regions.

Surface albedo represents the proportion of incoming solar radiation reflected by the surface under direct illumination conditions, computed for local solar noon. It depends on surface properties such as land cover, snow and ice extent, soil moisture, and vegetation characteristics. Surface albedo is essential for climate and weather studies, as it governs the amount of solar energy absorbed by the Earth, impacting energy exchange between the surface and atmosphere.

Surface albedo estimates are derived at a 10-day frequency using a 20-day moving temporal window of daily surface reflectance data. The processing algorithm normalises reflectances through inversion of a semi-empirical, kernel-driven reflectance model, prioritising recent observations. Directional albedo is computed per spectral band by integrating over the viewing hemisphere, with broadband albedo derived via a linear relationship from spectral albedo. Earlier versions of the dataset utilised data from NOAA/AVHRR (4km spatial resolution), SPOT-VGT, and PROBA-V (1km resolution), with products available from 1981 to the present. 



## Content of this notebook

This notebook explains how you can get access to EMCWF data and how you can download a custom-tailored subset of this dataset.

For the purpose of this assignment, only a specific variable and a delimited time span will be chosen, however this is scalable. 

The dataset has the following characteristics: 

* Data type: Gridded
* Projection: Plate Carrée
* Horizontal coverage: Global land surface
* Horizontal resolution: AVHRR: 1/30° (\~4 km), VGT: 1/112° (\~1 km), Sentinel-3: 1/336° (~300 m)
* Vertical coverage: Top of the canopy
* Temporal coverage: AVHRR: September 1981 to December 2005, SPOT-VGT: April 1998 to May 2014, PROBA-VGT: October 2013 to June 2020, Sentinel-3: July 2018 to December 2024
* Temporal resolution: 10 days
* File format: NetCDF

For more information read through the [Surface albedo documentation](https://cds.climate.copernicus.eu/datasets/satellite-albedo?tab=documentation).




## How to proceed

1. First register at [Copernicus Data Store](https://accounts.ecmwf.int/auth/realms/ecmwf/protocol/openid-connect/auth?client_id=cds&scope=openid%20email&response_type=code&redirect_uri=https%3A%2F%2Fcds.climate.copernicus.eu%2Fapi%2Fauth%2Fcallback%2Fkeycloak&state=eSFj1D686DwLjXr9KQRyDdS8UA1dAVnzsJzSvXQTetM&code_challenge=nf66xbC6c-X_rt1RIoT4al-fNscBETZhfSzP0i1DSbc&code_challenge_method=S256). Copy and store your key??

2. Install the CDS API. Use pip or anaconda, depending on your specific case: 
 ```
for Anaconda use : conda install -c conda-forge cdsapi)
 ```

 * Next, copy and store your API key in file `$HOME/.cdsapirc`. The key can be found at the bottom of your personal profile when you are logged in to the CDS. The file should be created in the HOME location. Format of the `.cdsapirc` file:
 ```
url: https://cds.climate.copernicus.eu/api
key: {your-api-key-here}
```

3. Browse through the [Albedo catalogue](https://cds.climate.copernicus.eu/datasets/satellite-albedo?tab=download) and select the fields you want to download. 
 * Use the API request code generator tool. 
 * Important: read through the [Albdedo documentation] (https://cds.climate.copernicus.eu/datasets/satellite-albedo?tab=documentation) to understand the data structure and identify which data/access point we need.

4. Copy the generated code lines to the notebook. Define Netcdf as output format.

5. Run the code and wait for the file to be downloaded. 

In [6]:
 
#local_lib = '/p/home/jusers/schultz1/jsccloud/.local/lib/python3.12/site-packages/'

#print(local_lib in sys.path)
#if not (local_lib in sys.path):
#    sys.path.append(local_lib)

import sys
import netCDF4 

print(f"Python executable path: {sys.executable}")
print(f"netCDF4 version: {netCDF4.__version__}")

Python executable path: C:\Users\nagib\anaconda3\python.exe
netCDF4 version: 1.5.7


In [2]:
import cdsapi

dataset = "satellite-albedo"
request = {
    "variable": ["albb_bh"],
    "satellite": ["noaa_14"],
    "sensor": "avhrr",
    "horizontal_resolution": ["4km"],
    "product_version": ["v2"],
    "year": ["1999"],
    "month": ["07"],
    "nominal_day": [
        "10",
        "20",
        "31"
    ],
    "area": [90, -180, -90, 180], 
     "format": "netcdf"
}

client = cdsapi.Client()
client.retrieve(dataset, request, 'Sentinel3_SurfaceAlbedo.nc')




2025-12-03 00:56:14,353 INFO Request ID is 7574fdae-0df2-4bf0-8967-9f80cbcc2f30
2025-12-03 00:56:14,453 INFO status has been updated to accepted
2025-12-03 00:56:28,104 INFO status has been updated to running
2025-12-03 00:56:47,255 INFO status has been updated to successful


6845c08fbc18bbbbe248ac020247035d.zip:   0%|          | 0.00/287M [00:00<?, ?B/s]

'Sentinel3_SurfaceAlbedo.nc'

In [12]:
## nc file seems to be zipped, then we have to uncompress it
import zipfile
import os

zip_filename = 'Sentinel3_SurfaceAlbedo.nc'

# Check if the downloaded file is a zip archive
if zipfile.is_zipfile(zip_filename):
    print(f"File {zip_filename} is a ZIP archive. Extracting...")
    
    # Create a ZipFile object and extract all contents
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.') # Extract into the current directory
        
        # Assume the actual .nc file is inside the zip with the same name or a similar name
        # You may need to manually check the content of the zip if this fails.
        
    print("Extraction complete. Attempting to find and open the .nc file.")
else:
    print(f"File {zip_filename} is not a zip archive. Proceeding with original name.")

File Sentinel3_SurfaceAlbedo.nc is a ZIP archive. Extracting...
Extraction complete. Attempting to find and open the .nc file.


In [13]:
import xarray as xr
try:
    ds = xr.open_dataset('c3s_ALBB-BH_19990710000000_GLOBE_AVHRR-NOAA14_V2.0.1.area-subset.90.180.-90.-180.nc', engine='netcdf4')
    print("Dataset opened successfully with 'netcdf4' engine.")
    print(ds)
except Exception as e:
    print(f"Failed to open with explicit 'netcdf4' engine: {e}")

Failed to open with explicit 'netcdf4' engine: [Errno -101] NetCDF: HDF error: b'C:\\Users\\nagib\\Documents\\Uni K\xc3\xb6ln\\2 Semester\\ESDP\\Homework\\earth-system-data-processing\\data_access\\c3s_ALBB-BH_19990710000000_GLOBE_AVHRR-NOAA14_V2.0.1.area-subset.90.180.-90.-180.nc'
